# Description:
  - This example shows how to retrieve data from Azure SQL DB by using Open AI GPT.
  - Asking questions in plain english that gets "translated" by GPT into SQL.
  - Using Langchain SQLDatabaseChain
  
## Requirements:
  - For this laboratory you need the Microsoft ODBC 18 driver installed in your system:
    - Windows: https://learn.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver16#download-for-windows
    - Mac: https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/install-microsoft-odbc-driver-sql-server-macos?view=sql-server-ver15#microsoft-odbc-18
    - Linux: https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver15&tabs=ubuntu18-install%2Cubuntu17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline#18

## Sample questions you can ask:
  - List the tables in the database
  - How many products are in the Adventure Works database?
  - How many Products are color black?
  - How many SalesOrderDetail are for the Product AWC Logo Cap ?
  - List the top 10 most expensive products

## For more information about Langchain agent toolkits, see:
  https://python.langchain.com/docs/modules/agents/tools/toolkits


In [1]:
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent, SQLDatabaseToolkit
from langchain.chains import create_sql_query_chain

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_GPT35_DEPLOYMENT_NAME = os.getenv("OPENAI_GPT35_DEPLOYMENT_NAME")
OPENAI_GPT4_DEPLOYMENT_NAME = os.getenv("OPENAI_GPT4_DEPLOYMENT_NAME")

SQL_SERVER = os.getenv("SQL_SERVER")
SQL_USER = os.getenv("SQL_USER")
SQL_PWD = os.getenv("SQL_PWD")
SQL_DBNAME = os.getenv("SQL_DBNAME")

In [2]:
def init_llm(model=OPENAI_GPT35_DEPLOYMENT_NAME,
             deployment_name=OPENAI_GPT35_DEPLOYMENT_NAME,
             openai_api_version="2024-02-15-preview",
             temperature=0,
             max_tokens=400
             ):

    llm = AzureChatOpenAI(deployment_name=deployment_name,
                            model=model,
                            openai_api_version=openai_api_version,
                            azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
                            temperature=temperature,
                            max_tokens=max_tokens
                            )
    return llm

llm = init_llm()

In [4]:
# Configuration for the database connection
db_config = {
    'drivername': 'mssql+pyodbc',
    'username': SQL_USER + '@' + SQL_SERVER,
    'password': SQL_PWD,
    'host': SQL_SERVER,
    'port': 1433,
    'database': SQL_DBNAME,
    'query': {'driver': 'ODBC Driver 18 for SQL Server'},
}

# Create a URL object for connecting to the database
db_url = URL.create(**db_config)

# Connect to the Azure SQL Database using the URL string
engine = create_engine(db_url)

# Test the connection using the SQLAlchemy 2.0 execution style
with engine.connect() as conn:
    try:
        # Use the text() construct for safer SQL execution
        result = conn.execute(text("SELECT @@VERSION"))
        version = result.fetchone()
        print("Connection successful!")
        print(version)
    except Exception as e:
        print(e)

db = SQLDatabase.from_uri(db_url, schema='SalesLT')

Connection successful!
('Microsoft SQL Azure (RTM) - 12.0.2000.8 \n\tFeb  2 2024 04:20:23 \n\tCopyright (C) 2022 Microsoft Corporation\n',)


In [5]:
chain = create_sql_query_chain(llm, db)

In [6]:
query = chain.invoke({"question":"How many products are in the Adventure Works database using SalesLT schema?"})
print("Query generated by OpenAI: " + query)

Query generated by OpenAI: SELECT COUNT(*) AS TotalProducts
FROM [SalesLT].[Product]


In [7]:
db.run(query)

'[(295,)]'

In [8]:
query = chain.invoke({"question":"How many Products are color black?"})
print("Query generated by OpenAI: " + query)

Query generated by OpenAI: SELECT COUNT(*) AS BlackProducts
FROM [SalesLT].[Product]
WHERE [Color] = 'Black'


In [9]:
db.run(query)

'[(89,)]'

In [10]:
query = chain.invoke({"question":"How many SalesOrderDetail are for the Product AWC Logo Cap ?"})
print("Query generated by OpenAI: " + query)

Query generated by OpenAI: SELECT COUNT(*) AS NumberOfSalesOrderDetail
FROM [SalesLT].[SalesOrderDetail] sod
JOIN [SalesLT].[Product] p ON sod.ProductID = p.ProductID
WHERE p.Name = 'AWC Logo Cap'


In [11]:
db.run(query)

'[(9,)]'

### Let's work with Agents to give some reasoning capabilities to our chatbot. 
GPT 4 is mandatory to run this section. GPT3.5 will make mistakes.

In [12]:
def init_llm(model=OPENAI_GPT4_DEPLOYMENT_NAME,
             deployment_name=OPENAI_GPT4_DEPLOYMENT_NAME,
             openai_api_version="2024-02-15-preview",
             temperature=0,
             max_tokens=400
             ):

    llm = AzureChatOpenAI(deployment_name=deployment_name,
                            model=model,
                            openai_api_version=openai_api_version,
                            azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
                            temperature=temperature,
                            max_tokens=max_tokens
                            )
    return llm

llm = init_llm()

In [13]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    top_k=30,
    agent_type="openai-tools",
    verbose=True
)

In [17]:
response = agent_executor.invoke("Give me the breakdown of sales by city and country. Only include the 10 cities with the highest sales in descending order.") 



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with ``


Address, Customer, CustomerAddress, Product, ProductCategory, ProductDescription, ProductModel, ProductModelProductDescription, SalesOrderDetail, SalesOrderHeader
Invoking: `sql_db_schema` with `{'table_names': 'SalesOrderHeader,Address'}`



CREATE TABLE [SalesLT].[Address] (
	[AddressID] INTEGER NOT NULL IDENTITY(1,1), 
	[AddressLine1] NVARCHAR(60) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[AddressLine2] NVARCHAR(60) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[City] NVARCHAR(30) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[StateProvince] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[CountryRegion] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[PostalCode] NVARCHAR(15) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	rowguid UNIQUEIDENTIFIER NOT NULL DEFAULT (newid()), 
	[ModifiedDate] DATETIME NOT NULL DEFAULT (getdate()), 
	CONSTRAINT [PK_Address_Addr

In [19]:
response = agent_executor.invoke("Dá-me as vendas por cidade e país. Inclui apenas as 10 cidades com as vendas mais altas por ordem decrescente.") 



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with ``


Address, Customer, CustomerAddress, Product, ProductCategory, ProductDescription, ProductModel, ProductModelProductDescription, SalesOrderDetail, SalesOrderHeader
Invoking: `sql_db_schema` with `{'table_names': 'SalesOrderHeader,Address'}`



CREATE TABLE [SalesLT].[Address] (
	[AddressID] INTEGER NOT NULL IDENTITY(1,1), 
	[AddressLine1] NVARCHAR(60) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[AddressLine2] NVARCHAR(60) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[City] NVARCHAR(30) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[StateProvince] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[CountryRegion] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[PostalCode] NVARCHAR(15) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	rowguid UNIQUEIDENTIFIER NOT NULL DEFAULT (newid()), 
	[ModifiedDate] DATETIME NOT NULL DEFAULT (getdate()), 
	CONSTRAINT [PK_Address_Addr